In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# In-depth tutorial: single cycle
This tutorial/example aims to go a bit more in-depth into the code-base compared to the "Hello World" example, and discuss the major classes and functions, and how they relate to one another. The outcome is the implementation of a single update loop.

## Muon generation
The `tomopt.muon` contains functions and classes to deal with the generation and handling of muons.

`tomopt.muon.generation.generate_batch` will generate N muons on demand via random sampling. These are stored as an (N,5) tensor, with columns corresponding to (x,y,momentum,$\theta_x$,$\theta_y$), where $\theta$ is the angle between the z-axis and the trajectory of the muon in x & y. Currently x & y are uniform in $[0,1)$, momentum is fixed, and the $\theta$ is not properly sampled (a scaled & clamped Gaussian is used).

In [ ]:
from tomopt.muon import generate_batch

In [ ]:
generate_batch(10)

To provide a more convenient interface, `tomopt.muon.muon_batch.MuonBatch` is used to wrap the generated muons with methods, and property getters and setters. When instantiating a `MuonBatch`, we also need to tell it where the muons start in z. Propagation of the muons proceeds in steps of $\delta z$ in the negative z direction.

In [ ]:
from tomopt.muon import MuonBatch

In [ ]:
muons = MuonBatch(generate_batch(1000), init_z=1)
muons

In [ ]:
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

In [ ]:
muons.propagate(dz=0.1)

In [ ]:
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

Normally, though, we let the volume layers (see next section) call the `propagate` method. As the muons pass through passive material and scatter the $\theta$ values of the muons will also change (along with x & y). Detector layers will append hits with reconstructed x & y to the `MuonBatch.hits` attribute, by calling `MuonBatch.append_hits`. After traversing the entire volume, the hits can be extracted using `MuonBatch.get_hits`.

## Volume definition

### Passive volume definition
First let's set up the volume; a block occupying (x,y,z) space from (0,0,0)->(L,W,H), and subdivided into cubic voxels of width *size*. Each voxel can be specified as a different material with varying x0 (radiation length [m]). `tomopt.core.X0` includes a dictionary of the x0 in various materials.

The `tomopt.volume` contains methods and classes to enable the definition of the active and passive volumes. Construction of a passive volume is done layer-wise in the z-axis, and each layer should be a `tomopt.volume.layer.PassiveLayer`. These are initialised by stating the z-position of the **top** of the layer, the transverse length and width of layer, and the size of each voxel (simultaneously defines the depth of the layer and the number of voxels in the layer). The user should ensure that the length and width are both divisible by the size. The materials of the voxels in the layer are defined using a function which takes the coordinates of the layer and returns an (N,M) tensor with the X0 of material for the (NxM voxels in x,y).

Current limitations:
- The scattering model used highly simplified.
- The computation of the number of radition lengths in a $\delta z$ step is based solely on the material of the voxel at the start of the step.

Below, we'll look at how to construct a passive layer

In [ ]:
import torch
from torch import Tensor

In [ ]:
from tomopt.core import X0

The function below takes the layer coordinates and returns a tensor with voxel material for the layer at the specified z, In this case, it will return beryllium for all voxels, except for the layer at z=0.4, which will contain a block of lead for x_voxels > 5 and y_voxels > 5. Note that the function takes absolute coordinates in metres and must manually convert to voxel-coordinates using the size.

In [ ]:
def arb_rad_length(*,z:float, lw:Tensor, size:float) -> float:
    rad_length = torch.ones(list((lw/size).long()))*X0['beryllium']
    if z >= 0.4 and z <= 0.5: rad_length[5:,5:] = X0['lead']
    return rad_length

In [ ]:
from tomopt.volume import PassiveLayer

In [ ]:
pl = PassiveLayer(rad_length_func=arb_rad_length, lw=Tensor([1,1]), z=0.2, size=0.1)

The `PassiveLayer` inherits from `tomopt.volume.layer.Layer`, which in turn inherits from `torch.nn.Module`, and passing a `MuonBatch` through a `Layer` involves calling its `forward` method (or calling the object, since `nn.Module.__call__` points to `forward`).

When a `MuonBatch` is passed through the `PassiveLayer`, it does so in `n` steps of $\delta z$, and at each step undergoes multiple scattering according the material traversed. Note that the `forward` method does not return the propagated `MuonBatch`, but instead updates the internal parameters of the `MuonBatch` in-place.

In [ ]:
muons = MuonBatch(generate_batch(1000), init_z=0.2)
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

In [ ]:
pl(mu=muons, n=2)

In [ ]:
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

Note that the muons have dropped to the bottom of the passive layer, and that their $\theta$ values have also changed due to the multiple-scattering. Multiple scattering can also cause shifts in x and y.

### Detector layer definition
Similar to the passive volume, detectors are build in layers of cubic voxels of width *size*. Every detector voxel *exists*, in order to prevent non-differenitial creation/destruction of detectors, but their resolution and efficiency are parameters to be optimised. The *cost* of each voxel depends on its resolution and efficiency and by including the cost in the loss function, the optimisation is prevented from simply maxing out the detector parameters.

Current limitations:
- Currently, TomOpt only supports the use of two pairs of detector layers, one pair above, and the other below the passive volume.
- Additionally, detector layers do **not** scatter muons.

Again `tomopt.volume.layer.VoxelDetectorLayer` is initialised using the z-position of the **top** of the layer, the transverse length and width of layer, and the size of each voxel (simultaneously defines the depth of the layer and the number of voxels in the layer). The user should ensure that the length and width are both divisible by the size. Additionally, the user must provide the initial resolution and efficiency, which will be used by all elements of the layer, and the functions for relating the resolution and efficiency values to the cost of the detector voxel.

Below are two arbitrary cost functions, remembering that the efficiency will be clamped between [0,1], and the resolution should typically be around 1e3-1e4 $m^{-1}$:

In [ ]:
import matplotlib.pyplot as plt

import torch.nn.functional as F

In [ ]:
def eff_cost(x:Tensor) -> Tensor:
    return torch.expm1(3*F.relu(x))  # free for negative efficiency, sharp rise as efficiency increases

x = torch.linspace(0,1,10)
plt.plot(x, eff_cost(x))

In [ ]:
def res_cost(x:Tensor) -> Tensor:
    return F.relu(x/100)**2  # free for negative resoltuion, gradual rise as resoltuion increases

x = torch.logspace(1,4,100)
plt.plot(x, res_cost(x))

Now we can build our detector layer. As mentioned before, TomOpt expects two pairs of detectors, one above and the other below. The `pos` argument takes strings and defines where which pair the layer will belong to.

In [ ]:
from tomopt.volume import VoxelDetectorLayer

In [ ]:
dl = VoxelDetectorLayer(pos='above', init_eff=0.5, init_res=1000, lw=Tensor([1,1]), z=1, size=0.1, eff_cost_func=eff_cost, res_cost_func=res_cost)

Similarly to `PassiveLayer` we pass muons through the layer by passing them to the `forward` method (or by calling the layer)

In [ ]:
muons = MuonBatch(generate_batch(1000), init_z=1)
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

In [ ]:
dl(mu=muons)

In [ ]:
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

Note that the muons move to the bottom of the layer, but no multiple scattering takes place (`theta` is unchanged). Instead hits have been recorded per muon:

In [ ]:
muons.hits

The xy coordinates are the positions of the muons as recorded by the detector voxel the muon passed through, and is computed according to: $x_{\mathrm{reco}}=\mathcal{N}\left(x_{\mathrm{true}},\mathrm{res}^{-1}\right)$, however the recorded positions are clamped to stay within the voxel that the muon passed through. There are several things to be aware of:
- Muons outside the detector still have hits recorded for them. This is for simplicity when propagating the muon batch through the volume, and muons that exit the volume are later filtered out
- All muons inside the detector cause hits regardless of the detector efficiency. Throwing a random number against the efficiency does not allow the loss to be differentiated w.r.t the efficiency, instead the product of all the efficiencies encountered by a given muon is used to weight its contributions to the prediction of the x0.

Eventually, each muon will accumulate 4 hits. `MuonBatch.get_hits` concatenates the hits together for muons which are still inside the detector at the end of the propagation:

In [ ]:
muons.get_hits(lw=Tensor([1,1]))

### Building the whole volume
Now we can build both passive and detector layers, we can build the full volume by stacking layers together in z, remembering that we need two pairs of detector layers. Below we write a function to return a 1x1x1 m cube with 10 layers (size=0.1m), with the layers stored in a `torch.nn.ModuleList`:

In [ ]:
import numpy as np
from torch import nn

In [ ]:
def get_layers():
    layers = []
    lwh = Tensor([1,1,1])
    size = 0.1
    init_eff = 0.5
    init_res = 1000
    pos = 'above'
    for z,d in zip(np.arange(lwh[2],0,-size), [1,1,0,0,0,0,0,0,1,1]):
        if d:
            layers.append(VoxelDetectorLayer(pos=pos, init_eff=init_eff, init_res=init_res,
                                        lw=lwh[:2], z=z, size=size, eff_cost_func=eff_cost, res_cost_func=res_cost))
        else:
            pos = 'below'
            layers.append(PassiveLayer(rad_length_func=arb_rad_length, lw=lwh[:2], z=z, size=size))

    return nn.ModuleList(layers) 

In [ ]:
get_layers()

For convenience, we store this in a `tomopt.volume.volume.Volume`, which provides a variety of additional methods, and during the `forward` method will pass the muons through each layer in turn.

In [ ]:
from tomopt.volume import Volume

In [ ]:
volume = Volume(get_layers())
volume

We can grab passive layers and detectors via:

In [ ]:
volume.get_detectors()

In [ ]:
volume.get_passives()

Build a tensor of the x0 in the passive volume:

In [ ]:
x0 = volume.get_rad_cube()
x0.shape, x0

We can lookup absolute coordinates and get the voxel indices:

In [ ]:
volume.lookup_passive_xyz_coords(xyz=torch.tensor([[0.5,0.2,0.2],[0.3,0.8,0.7]], device=volume.device))

And we can replace the entire x0 composition of the passive volume with a new function:

In [ ]:
volume.load_rad_length(arb_rad_length)

We can pass muons through the entire volume by simply calling the `forward` method of the `Volume`, which will pass the muon batch though each layer in turn:

In [ ]:
muons = MuonBatch(generate_batch(1000), init_z=1)
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

In [ ]:
volume(mu=muons)

In [ ]:
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

We can also access the hits in the four detector layers:

In [ ]:
hits = muons.get_hits(volume.lw)
hits

In [ ]:
hits['above']['reco_xy'].shape  # (muons, detector layer, xy)

## Scattering inference
Now that we can construct volumes and pass muons through them, the next step is to infer the composition of the passive volume from the scattering of the muons. So far in TomOpt, the only method implemented uses the Point Of Closest Approach (POCA). This involves extrapolating straight lines from the two pairs of hits in the detector layers above and below the passive volume. The difference in $\theta$ for these lines is due to the multiple scattering across all of the passive layers, however as a simplification, the entirety of the multiple scattering is assumed to have occurred at the point of closest approach of the two lines; i.e. every muon only provides information for a single point, and that information is biased to underestimate x0.

The `tomopt.inference.scattering.VoxelScatterBatch` class is initialised using the muon batch after propagation and the volume through which it was propagated. For every muon that stays within the volume, it will compute in absolute units:
- `location`: The location of the point of closest approach. (muons, x, y, z).
- `dtheta`: The difference in $\theta$ of the extrapolated muons trajectories, i.e. the shift in angle due to multiple scattering. (muons, $\delta\theta_x$, $\delta\theta_y$).
- `dxy`: The length of the normal vector between the extrapolated muons trajectories, i.e. the shift in xy due to multiple scattering. (muons, $\delta x$, $\delta y$).
- `theta_in` & `theta_out`: The estimated $\theta$ of the incoming & outgoing muons. (muons, $\theta_x$, $\theta_y$).
- `*_unc`: The uncertainties associated with the above quantities due to the resolution associated with the hits. These are computed via auto-differentiation and can be quite slow.

In [ ]:
from tomopt.inference import VoxelScatterBatch

In [ ]:
muons = MuonBatch(generate_batch(100), init_z=1)
volume = Volume(get_layers())
volume(muons)

In [ ]:
%%time
sb = VoxelScatterBatch(mu=muons, volume=volume)

In [ ]:
%%time
sb.location, sb.location_unc

In [ ]:
%%time
sb.dtheta, sb.dtheta_unc

## X0 inference
From the scattering information, the next step is to use the information to infer the x0 of the passive volume. This is achieved by inverting the scattering equations to put them in terms of x0, however currently only the $\theta$ scattering formula has been inverted. N.B. the scattering formulas include random terms whose means of squares are one, however this relies on a sufficient number of muons **per voxel** in order to provide an accurate inference of x0. This, combined with the biased computation of $\delta\theta$ and the uncertainty due to resolution on scattering properties, means that x0 inference can be expected to be both inaccurate and imprecise.

To account for the fact that the inferred scatter location is uncertain and predictions of each muon carry different uncertainties, the x0 prediction of each muon goes into a weighted average for every voxel. The weights of each prediction per voxel account for the efficiency of the detectors the muon passed through, the uncertainty in the x0 prediction, and the probability of the scattering occurring in the given voxel according to the inferred scatter location and its uncertainty.

`tomopt.inference.rad_length.VoxelX0Inferer` is used to compute predictions of the x0 composition of the passive volume from the `VoxelScatterBatch` results.

In [ ]:
from tomopt.inference import VoxelX0Inferer

In [ ]:
x0_inferer = VoxelX0Inferer(scatters=sb)

Currently the only inference method implemented is `x0_from_dtheta`. This computes the x0 predicted per muon, along with their uncertainties:

In [ ]:
%%time
pred, pred_unc = x0_inferer.x0_from_dtheta()

In [ ]:
pred[0], pred_unc[0]

In [ ]:
from tomopt.utils import jacobian
jacobian(pred_unc, volume.get_detectors()[0].resolution, create_graph=True).sum((-2,-1))[:10]

The more general method `pred_x0` will call `x0_from_dtheta` and `x0_from dxy` (when implemented) and then compute a weighted average of the x0 predictions per voxel, returning a rank-3 tensor with the same number of elements as the passive volume. The weights used for per-muon ($i$) per voxel ($j$) predictions are:
$$w_{i,j} = \frac{\epsilon_i\times p_{i,j}}{\alpha_{x_{0,i}}^2},$$
where $\epsilon$ is the product of the efficiencies of the four hits associated with muon $i$, $\alpha_{x_{0,i}}$ is the uncertainty associated with prediction of muon $i$, and $p_{i,j}$ is the integral in x,y,z over voxel $j$ of a mulitivariate Gaussian (uncorrelated) centred at the scatter location of muon $i$ and widths equal to the uncertainty of the scatter location in x,y,z. Since these weights are also useful for constructing losses, the sum of weights per voxel is also returned.

In [ ]:
pred, weight = x0_inferer.pred_x0()

In [ ]:
pred

## Loss
So far only a single loss class is implemented in TomOpt: `tomopt.optimisation.loss.loss.DetectorLoss`. This consists of two components: The precision component based on the square error multiplied by the variance, averaged over the voxels $\left< \left(x_{0,\mathrm{pred}}-x_{0,\mathrm{true}}\right)^2 / w\right>$; And the cost of the whole detector timesed by a scaling coefficient.

In [ ]:
from tomopt.optimisation import DetectorLoss

In [ ]:
loss_func = DetectorLoss(target_budget=0.8, cost_coef=0)

In [ ]:
loss_val = loss_func(pred_x0=pred, pred_weight=weight, volume=volume)
loss_val

Let's backprop the loss and check that the detector parameters accumulate gradients:

In [ ]:
volume.get_detectors()[0].resolution.grad, volume.get_detectors()[0].efficiency.grad

In [ ]:
loss_val.backward()

In [ ]:
volume.get_detectors()[0].resolution.grad, volume.get_detectors()[0].efficiency.grad

So generally increasing the resolution and increasing the efficiency should decrease the loss. Additionally, there are sometimes NaN gradients. I'm not sure what causes these at the moment, but we can replace them zeros for now.

In [ ]:
for l in volume.get_detectors():
    torch.nan_to_num_(l.resolution.grad, 0)
    torch.nan_to_num_(l.efficiency.grad, 0)

## Optimiser
TomOpt uses the optimisers built into PyTorch, however due to the difference in scales for the resolution and efficiencies, we instead use two separate optimisers:

In [ ]:
from torch import optim

In [ ]:
res_opt = optim.SGD((l.resolution for l in volume.get_detectors()), lr=2e10)
eff_opt = optim.SGD((l.efficiency for l in volume.get_detectors()), lr=2e5)

In [ ]:
volume.get_detectors()[0].resolution, volume.get_detectors()[0].efficiency

In [ ]:
res_opt.step()
eff_opt.step()

In [ ]:
volume.get_detectors()[0].resolution, volume.get_detectors()[0].efficiency

So the parameters shifted in the way that we expected from the gradients